# 1204. Last Person to Fit in the Bus

SQL SchemaPandas SchemaTable: Queue+-------------+---------+| Column Name | Type    |+-------------+---------+| person_id   | int     || person_name | varchar || weight      | int     || turn        | int     |+-------------+---------+person_id column contains unique values.This table has the information about all people waiting for a bus.The person_id and turn columns will contain all numbers from 1 to n, where n is the number of rows in the table.turn determines the order of which the people will board the bus, where turn=1 denotes the first person to board and turn=n denotes the last person to board.weight is the weight of the person in kilograms. There is a queue of people waiting to board a bus. However, the bus has a weight limit of 1000 kilograms, so there may be some people who cannot board.Write a solution to find the person_name of the last person that can fit on the bus without exceeding the weight limit. The test cases are generated such that the first person does not exceed the weight limit.Note that only one person can board the bus at any given turn.The result format is in the following example. **Example 1:**Input: Queue table:+-----------+-------------+--------+------+| person_id | person_name | weight | turn |+-----------+-------------+--------+------+| 5         | Alice       | 250    | 1    || 4         | Bob         | 175    | 5    || 3         | Alex        | 350    | 2    || 6         | John Cena   | 400    | 3    || 1         | Winston     | 500    | 6    || 2         | Marie       | 200    | 4    |+-----------+-------------+--------+------+Output: +-------------+| person_name |+-------------+| John Cena   |+-------------+Explanation: The folowing table is ordered by the turn for simplicity.+------+----+-----------+--------+--------------+| Turn | ID | Name      | Weight | Total Weight |+------+----+-----------+--------+--------------+| 1    | 5  | Alice     | 250    | 250          || 2    | 3  | Alex      | 350    | 600          || 3    | 6  | John Cena | 400    | 1000         | (last person to board)| 4    | 2  | Marie     | 200    | 1200         | (cannot board)| 5    | 4  | Bob       | 175    | ___          || 6    | 1  | Winston   | 500    | ___          |+------+----+-----------+--------+--------------+

## Solution Explanation
This problem asks us to find the last person who can board a bus without exceeding the weight limit of 1000 kg. People board the bus in the order specified by the "turn" column.The approach is straightforward:1. Sort the queue by the "turn" column to get the boarding order2. Iterate through the sorted queue, keeping a running sum of weights3. For each person, add their weight to the running sum4. If the running sum exceeds 1000 kg, the previous person was the last one who could board5. If we go through all people without exceeding the limit, the last person in the queue is the answerFor SQL, we'll use a window function to calculate the cumulative sum of weights as people board in order. Then we'll find the person with the highest turn value whose cumulative weight doesn't exceed 1000 kg.

In [ ]:
import pandas as pddef queue_at_school(queue: pd.DataFrame) -> pd.DataFrame:    # Sort the queue by turn to get the boarding order    queue = queue.sort_values(by='turn')        # Initialize variables to track cumulative weight and last person to board    cumulative_weight = 0    last_person = None        # Iterate through each person in the queue    for _, person in queue.iterrows():        # Add the current person's weight to the cumulative weight        cumulative_weight += person['weight']                # If the cumulative weight exceeds 1000 kg, break the loop        if cumulative_weight > 1000:            break                    # Otherwise, update the last person who can board        last_person = person['person_name']        # Return the result as a DataFrame with a single column    return pd.DataFrame({'person_name': [last_person]})SQL solution:sqlWITH CumulativeWeight AS (    SELECT         person_name,        turn,        SUM(weight) OVER (ORDER BY turn ASC) AS total_weight    FROM Queue)SELECT person_nameFROM CumulativeWeightWHERE total_weight <= 1000ORDER BY turn DESCLIMIT 1;

## Time and Space Complexity
Time Complexity:* Sorting the queue by turn: O(n log n) where n is the number of people in the queue* Iterating through the sorted queue: O(n)* Overall time complexity: O(n log n)Space Complexity:* O(1) extra space for the cumulative weight and last person variables* The result DataFrame uses O(1) space as it contains only one entry* Overall space complexity: O(1)For the SQL solution:* Time Complexity: O(n log n) due to the sorting operation* Space Complexity: O(n) for the CumulativeWeight CTE

## Test Cases


In [ ]:
import pandas as pdimport unittestdef queue_at_school(queue: pd.DataFrame) -> pd.DataFrame:    queue = queue.sort_values(by='turn')    cumulative_weight = 0    last_person = None        for _, person in queue.iterrows():        cumulative_weight += person['weight']        if cumulative_weight > 1000:            break        last_person = person['person_name']        return pd.DataFrame({'person_name': [last_person]})class TestQueueAtSchool(unittest.TestCase):    def test_example_case(self):        data = {            'person_id': [5, 4, 3, 6, 1, 2],            'person_name': ['Alice', 'Bob', 'Alex', 'John Cena', 'Winston', 'Marie'],            'weight': [250, 175, 350, 400, 500, 200],            'turn': [1, 5, 2, 3, 6, 4]        }        queue = pd.DataFrame(data)        result = queue_at_school(queue)        self.assertEqual(result.iloc[0, 0], 'John Cena')        def test_all_can_board(self):        data = {            'person_id': [1, 2, 3, 4],            'person_name': ['Alice', 'Bob', 'Charlie', 'David'],            'weight': [200, 200, 200, 200],            'turn': [1, 2, 3, 4]        }        queue = pd.DataFrame(data)        result = queue_at_school(queue)        self.assertEqual(result.iloc[0, 0], 'David')  # Last person can board        def test_only_first_can_board(self):        data = {            'person_id': [1, 2, 3],            'person_name': ['Alice', 'Bob', 'Charlie'],            'weight': [800, 300, 200],            'turn': [1, 2, 3]        }        queue = pd.DataFrame(data)        result = queue_at_school(queue)        self.assertEqual(result.iloc[0, 0], 'Alice')  # Only first person can board        def test_exact_weight_limit(self):        data = {            'person_id': [1, 2, 3],            'person_name': ['Alice', 'Bob', 'Charlie'],            'weight': [400, 600, 200],            'turn': [1, 2, 3]        }        queue = pd.DataFrame(data)        result = queue_at_school(queue)        self.assertEqual(result.iloc[0, 0], 'Bob')  # Bob makes it exactly 1000kgif __name__ == '__main__':    unittest.main(argv=['first-arg-is-ignored'], exit=False)